In [5]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
query_embedding = model.encode('How big is London')
passage_embedding = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))

In [7]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests

with open("../data_test.json",'r') as f:
	source_data=json.load(f)

api_key = ''

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

In [ ]:
cap_attr_dict={}
for jsondata in tqdm(source_data[:100]):
	caption=jsondata['caption_L2L3']
	path='../vl_spec/'+jsondata['img_id']+'.vl.json'
	with open (path,'r') as f:
		vega=json.load(f)
	url="https://raw.githubusercontent.com/vis-nlp/Chart-to-text/main/statista_dataset/dataset/data/"+vega["data"]['url'][5:-4]+'.csv'
	attr=pd.read_csv(url).iloc[:, 0].astype(str).tolist()
	#Classify with GPT
	data = {
    "model": "gpt-3.5-turbo",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant for labeling datasets. Return only the label without explanation."},
        {"role": "user", "content": "You need to label data facts with different tasks and retrieve key values or words. The tasks include: HIGHLIGHT_ONE (value or word), COMPARE_TWO (value1, value2 or word1, word2), ADD_THRESHOLD (value), LOCAL_TREND (value1 and value2), GLOBAL_TREND (no value, no word)."},
        {"role": "assistant", "content": "I understand. I will just classify."},
        {"role": "user", "content": "The ticket price shown for 2006 is just over 60 dollars, and it rises from that point to just over 100 dollars in 2018."},
        {"role": "assistant", "content": "COMPARE_TWO, 2006, 2018"},
        {"role": "user", "content": "Viewers of Minecraft on twitch has gradually increased between 2018 and 2020."},
        {"role": "assistant", "content": "LOCAL_TREND, 2018, 2020"},
        {"role": "user", "content": "There are thousands of Bayern Munich fans and this number has seen a steady growth over the last twelve years."},
        {"role": "assistant", "content": "GLOBAL_TREND"},
        {"role": "user", "content": "The majorly of the big cities are at 3 index points."},
        {"role": "assistant", "content": "ADD_THRESHOLD, 3"},
        {"role": "user", "content": "Omsk has the largest value of 3.5 index points."},
        {"role": "assistant", "content": "HIGHLIGHT_ONE, Omsk"},
        {"role": "user", "content": caption}, # Using the target message here
    ],
    "temperature": 0
}
	response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))
	
	if response.status_code == 200:
		response_data = response.json()
		parsed_reponses=response_data['choices'][0]['message']['content'].split(', ')
		if len(parsed_reponses)>1:
			temp_dict={}
			for resp in parsed_reponses[1:]:
				query_embedding = model.encode(resp)
				passage_embedding = model.encode(attr)
				temp_dict[resp]=[attr[np.argmax(util.dot_score(query_embedding, passage_embedding).tolist()[0])],attr]
			cap_attr_dict[caption]=temp_dict
	else:
		print(f"Error: {response.status_code}")

with open("temp_dict.json",'w+') as f:
	json.dump(cap_attr_dict,f,indent=2)